In [1]:
import numpy as np;
import pandas as pd;

## Initialize Parameters

In [2]:
## parameters and global variables

mu = 0.01
district = "ksg"

## Load and Initialize Data

In [3]:
## Load population data
initDataDF = pd.read_csv("../data/" + district + "_init_data.csv")

## Load distance data
distanceData = pd.read_csv("../data/" + district + "_distance_matrix.csv")

## Number of regions
r = len(initDataDF.index)

## Sort population data and distance data by name for easy matrix transformation
initDataDF.sort_values(by=['name'], inplace=True)
distanceData.sort_values(by=['fromName', 'toName'], inplace=True)

## Prepare Distance matrix

In [4]:
distanceMatrix = distanceData['value'].to_numpy().reshape(r,r)
#distanceMatrix[0][4]

## Function Definitions

In [5]:
def getSij(i, j, distanceMatrix, initDataDF):
    #print(distanceMatrix[i] <= distanceMatrix[i][j])
    return(initDataDF[distanceMatrix[i] <= distanceMatrix[i][j]]['S'].sum() - initDataDF.iloc[i]['S'] - initDataDF.iloc[j]['S'])
    
#getSij(63,23,distanceMatrix, initDataDF)

In [6]:
def getTij(i,j, distanceMatrix, initDataDF):
    Ni = initDataDF.iloc[i]['S']
    Ti = mu*Ni

    if(i==j):
        return Ni - Ti;
    
    Nj = initDataDF.iloc[j]['S']
    Sij = getSij(i,j,distanceMatrix, initDataDF)
    Tij = Ti * ((Ni * Nj) /((Ni + Sij)*(Ni + Nj + Sij)))
    return Tij
    
    

## Compute TMatrix

In [7]:
data = []

for i in range(0,r):
    for j in range(0,r):
        Tij = getTij(i,j,distanceMatrix, initDataDF)
        #print("(", i, ",", j, ") ",  initDataDF.iloc[i]['name'], " ", initDataDF.iloc[j]['name'], " " ,Tij)
        data.append({'iName': initDataDF.iloc[i]['name'], 'jName': initDataDF.iloc[j]['name'], 'Tij': Tij});
        
resultsDF = pd.DataFrame(data);


## Save output

In [8]:
resultsDF.to_csv("../output/" + district + "_tmatrix_results.csv")